In [1]:
import warnings
warnings.simplefilter(action='ignore')

import os
import datetime, pandas

from tqdm import tqdm
from dataclasses import dataclass, asdict

import polars as pl
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple, Union
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import TimeSeriesSplit
import warnings
from abc import ABC, abstractmethod

import kaggle_evaluation.default_inference_server

from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoLars
from sklearn.linear_model import Lasso
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [2]:
train = pandas.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv').dropna()
test = pandas.read_csv('/kaggle/input/hull-tactical-market-prediction/test.csv').dropna()     

In [3]:
train = train[['E1','E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19',
       'E2', 'E20', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9',
        "S2", "P9", "S1", "S5", "I2", "P8",
        "P10", "P12", "P13", "forward_returns"
    ]]

for i in zip(train.columns, train.dtypes):
    train[i[0]].fillna(0, inplace=True)

train_split, val_split = train_test_split(
    train, test_size=0.01, random_state=42
)

X_train = train_split.drop(columns=["forward_returns"])
X_test = val_split.drop(columns=["forward_returns"])
y_train = train_split['forward_returns']
y_test = val_split['forward_returns']

In [4]:
X_train

,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,...,E9,S2,P9,S1,S5,I2,P8,P10,P12,P13
7206,1.041918,0.581349,0.006283,0.006283,0.006283,0.006283,0.006944,0.661232,1.223676,1.541528,...,0.043981,0.512567,0.033069,1.973754,-0.270671,-0.099589,2.057248,2.023172,-0.376245,0.192791
7034,0.615708,0.526124,0.007275,0.007275,0.007275,0.007275,0.006283,1.671331,1.929413,1.633859,...,0.004630,0.646952,0.000661,1.549308,0.135541,-1.652801,1.970050,1.914912,-0.151011,0.007275
7093,0.556396,0.815807,0.014220,0.001653,0.001653,0.001653,0.087963,1.849079,1.851411,1.912947,...,0.005622,-0.431026,0.124339,0.619904,-1.416954,-0.697070,1.898016,2.164918,-0.580531,0.384259
7588,2.446270,0.100860,0.029101,0.003307,0.016534,0.003307,0.016534,-0.901907,-0.926418,-0.211278,...,0.028770,0.237412,0.011574,-1.512129,-0.444974,-2.400884,1.706585,2.116923,-0.447031,0.756944
7552,2.141896,0.381614,0.016204,0.001653,0.001653,0.001653,0.000992,-1.894722,-1.788686,-0.766827,...,0.019841,1.748711,0.071429,0.143877,-0.059616,-2.082251,1.625190,1.983755,-0.838070,0.918651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8099,1.345860,0.773810,0.006944,0.006944,0.006944,0.006944,0.441468,-0.028077,0.449144,0.903182,...,0.620370,-1.953881,0.001984,0.895260,1.715770,-0.181981,2.318440,2.141596,0.602239,0.218585
8263,1.382723,0.834656,0.015873,0.002646,0.002646,0.002646,0.461310,-1.329866,-1.088179,-0.964261,...,0.316468,-1.127766,0.016534,0.298183,-0.813049,3.048313,1.186474,1.541084,-0.729057,0.116402
7829,1.347717,0.123677,0.051587,0.019841,0.019841,0.006614,0.432870,1.488317,1.584364,1.726731,...,0.715939,1.116874,0.082011,0.917571,0.089949,-2.151921,1.274399,1.610140,-0.781115,0.781415
8428,1.472010,0.515212,0.003638,0.003638,0.029431,0.003638,0.715939,-1.571494,-1.598439,-1.271266,...,0.484127,1.828487,0.060185,-0.644011,-0.322694,1.712999,2.295641,2.219362,-0.710188,0.669643


In [5]:
LGBM_R_parm = {'colsample_bytree': 0.7988997727163004,
                            'drop_rate': 0.2968017716958511,
                            'learning_rate': 0.17403795838781744,
                            'max_bin': 2707,
                            'max_depth': 9680,
                            'max_drop': 4736,
                            'min_child_samples': 7173,
                            'min_data_in_leaf': 458,
                            'n_estimators': 1655,
                            'num_leaves': 2755,
                            'objective': 'regression_l1',
                            'reg_alpha': 0.965759263160616,
                            'reg_lambda': 0.9274407181952318,
                            'skip_drop': 0.37396662816136594,
                            'verbosity': -1}
        
XGB_R_parm = {'colsample_bytree': 0,
                           'gamma': 5,
                           'learning_rate': 0.18348831817680378,
                           'max_depth': 15,
                           'min_child_weight': 1,
                           'n_estimators': 13225,
                           'objective': 'reg:squarederror',
                           'reg_alpha': 94,
                           'reg_lambda': 0.41318910368801975,
                           'subsample': 0.5444965693077323}

catboost_params = {'iterations' : 3000,
                                'learning_rate': 0.009, 
                                'depth': 5, 
                                'l2_leaf_reg': 5.5,
                                'min_child_samples' : 102,
                                'od_wait' : 50,
                                'random_state' : 42,
                                'eval_metric': 'RMSE', 
                                'od_type' : 'Iter',
                                'bootstrap_type': 'Bayesian', 
                                'grow_policy' : 'Depthwise',
                                'logging_level' : 'Silent'}

R_Forest_parm = {'n_estimators' : 25, 
                              'min_samples_split' : 2, 
                              'max_depth' : 10, 
                              'min_samples_leaf' : 2, 
                              'random_state' : 42}
        
Extra_parm = {'n_estimators' : 50, 
                           'min_samples_split' : 2, 
                           'max_depth' : 8, 
                           'min_samples_leaf' : 2, 
                           'random_state' : 42}
        
GB_params = {'learning_rate' : 0.1, 
                          'min_samples_split' : 500,
                          'min_samples_leaf' : 50,
                          'max_depth' : 8,
                          'max_features' : 'sqrt',
                          'subsample' : 0.8,
                          'random_state' : 10}

In [6]:
model_collecter = {}

LGBMR = LGBMRegressor(**LGBM_R_parm)
XGBR = XGBRegressor(**XGB_R_parm)
CatBoost = CatBoostRegressor(**catboost_params)

random_forest = RandomForestRegressor(**R_Forest_parm)
extra_trees = ExtraTreesRegressor(**Extra_parm)
GradientBoosting = GradientBoostingRegressor(**GB_params)

In [7]:
estimators_1 = [('LGBMR', LGBMR), ('XGBR', XGBR), ('CatBoost', CatBoost)]
estimators_2 = [('random_forest', random_forest), ('extra_trees', extra_trees), ('GradientBoosting', GradientBoosting)]
        
model_0 = StackingRegressor(estimators_1 + estimators_2, final_estimator = ElasticNetCV())
model_1 = StackingRegressor(estimators_1, final_estimator = RidgeCV())
model_2 = StackingRegressor(estimators_2, final_estimator = LassoLars())

estimators_3 = [('STACK_model_1', model_1),('STACK_model_2', model_2)]
model_3 = StackingRegressor(estimators_3, final_estimator = LassoCV())
        
estimators_final = [('STACK_model_0', model_0),('STACK_model_3', model_3)]
model_final = StackingRegressor(estimators_final, final_estimator = RidgeCV())
model_final.fit(X_train, y_train)

StackingRegressor(estimators=[('STACK_model_0',
                               StackingRegressor(estimators=[('LGBMR',
                                                              LGBMRegressor(colsample_bytree=0.7988997727163004,
                                                                            drop_rate=0.2968017716958511,
                                                                            learning_rate=0.17403795838781744,
                                                                            max_bin=2707,
                                                                            max_depth=9680,
                                                                            max_drop=4736,
                                                                            min_child_samples=7173,
                                                                            min_data_in_leaf=458,
                                                                            n_estimators=1655,
                                                                            num_leaves=2755,
                                                                            objective='regression_l1',
                                                                            reg_a...
                                                                                                                   random_state=42)),
                                                                                            ('extra_trees',
                                                                                             ExtraTreesRegressor(max_depth=8,
                                                                                                                 min_samples_leaf=2,
                                                                                                                 n_estimators=50,
                                                                                                                 random_state=42)),
                                                                                            ('GradientBoosting',
                                                                                             GradientBoostingRegressor(max_depth=8,
                                                                                                                       max_features='sqrt',
                                                                                                                       min_samples_leaf=50,
                                                                                                                       min_samples_split=500,
                                                                                                                       random_state=10,
                                                                                                                       subsample=0.8))],
                                                                                final_estimator=LassoLars()))],
                                                 final_estimator=LassoCV()))],
                  final_estimator=RidgeCV())

In [8]:
def predict(test: pl.DataFrame) -> float:
    test = test.to_pandas().drop(columns=["lagged_forward_returns", "date_id", "is_scored"])
    test = test[['E1','E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19',
       'E2', 'E20', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9',
        "S2", "P9", "S1", "S5", "I2", "P8",
        "P10", "P12", "P13"
    ]]                                                                                                                     
    for i in zip(test.columns, test.dtypes):
        test[i[0]].fillna(0, inplace=True)
    raw_pred = model_final.predict(test)[0]
    return raw_pred

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))